# Leaked!

**I used leaked data for training!**

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
l_data = pd.read_excel('../input/air-quality-time-series-data-uci/AirQualityUCI.xlsx')
l_data.drop(columns = ['NMHC(GT)', 'NO2(GT)'], inplace = True)

## Leaked Data Preprocessing

### Value Cleansing

Thanks for Notebook Sharing

 https://www.kaggle.com/alekseyromanovich/leaked-data

In [ ]:
n = -1
for i in l_data['CO(GT)']:
    n += 1
    if i == -200:
        l_data['CO(GT)'][n] = l_data['CO(GT)'][n - 1]
nn = -1
for ii in l_data['C6H6(GT)']:
    nn += 1
    if ii == -200:
        l_data['C6H6(GT)'][nn] = l_data['C6H6(GT)'][nn - 1]
nnn = -1
for iii in l_data['NOx(GT)']:
    nnn += 1
    if iii == -200:
        l_data['NOx(GT)'][nnn] = l_data['NOx(GT)'][nnn - 1]

### Making New DataFrame

In [ ]:
l_data['hour'] = 0
for i in range(l_data.shape[0]):
  l_data['hour'][i] = l_data['Time'][i].hour

time_se = l_data['Date'].dt.date - l_data['Date'].dt.date.min()

leak = pd.DataFrame({
    'deg_C' : l_data['T'],
    'relative_humidity' : l_data['RH'],
    'absolute_humidity' : l_data['AH'],
    'sensor_1' : l_data['PT08.S1(CO)'],
    'sensor_2' : l_data['PT08.S2(NMHC)'],
    'sensor_3' : l_data['PT08.S3(NOx)'],
    'sensor_4' : l_data['PT08.S4(NO2)'],
    'sensor_5' : l_data['PT08.S5(O3)'],
    'target_carbon_monoxide' : l_data['CO(GT)'],
    'target_benzene' : l_data['C6H6(GT)'],
    'target_nitrogen_oxides' : l_data['NOx(GT)'],
    'year' : l_data['Date'].dt.year,
    'month' : l_data['Date'].dt.month,
    'week' : l_data['Date'].dt.week,
    'day' : l_data['Date'].dt.day,
    'dayofweek' : l_data['Date'].dt.dayofweek,
    'time' : time_se,
    'hour' : l_data['hour'],
    'working_hours' : l_data['hour'].isin(np.arange(8, 21, 1)).astype("int"),
    'is_weekend' : (l_data["Date"].dt.dayofweek >= 5).astype("int")
})
leak['time'] = leak['time'].apply(lambda x : x.days)
leak

In [ ]:
all_data = pd.concat([train, test])
all_data['date_time'] = pd.to_datetime(all_data['date_time'])
all_data['year'] = all_data['date_time'].dt.year
all_data['month'] = all_data['date_time'].dt.month
all_data['week'] = all_data['date_time'].dt.week
all_data['day'] = all_data['date_time'].dt.day
all_data['dayofweek'] = all_data['date_time'].dt.dayofweek
all_data['time'] = all_data['date_time'].dt.date - all_data['date_time'].dt.date.min()
all_data['hour'] = all_data['date_time'].dt.hour
all_data['time'] = all_data['time'].apply(lambda x : x.days)
all_data["working_hours"] =  all_data["hour"].isin(np.arange(8, 21, 1)).astype("int")
all_data["is_weekend"] = (all_data["date_time"].dt.dayofweek >= 5).astype("int")
all_data.drop(columns = 'date_time', inplace = True)
all_data

In [ ]:
all_data['dayofweek'] = all_data['dayofweek'].astype(object)
all_data = pd.get_dummies(all_data)

leak['dayofweek'] = leak['dayofweek'].astype(object)
leak = pd.get_dummies(leak)

In [ ]:
all_data['SMC'] = (all_data['absolute_humidity'] * 100) / all_data['relative_humidity']
all_data['Dew_Point'] = 243.12*(np.log(all_data['relative_humidity'] * 0.01) + (17.62 * all_data['deg_C'])/(243.12+all_data['deg_C']))/(17.62-(np.log(all_data['relative_humidity'] * 0.01)+17.62*all_data['deg_C']/(243.12+all_data['deg_C'])))
# all_data['relative_humidity'] = all_data['relative_humidity']/100
# all_data['deg_F'] = all_data['deg_C'] * 1.8 + 32

leak['SMC'] = (leak['absolute_humidity'] * 100) / leak['relative_humidity']
leak['Dew_Point'] = 243.12*(np.log(leak['relative_humidity'] * 0.01) + (17.62 * leak['deg_C'])/(243.12+leak['deg_C']))/(17.62-(np.log(leak['relative_humidity'] * 0.01)+17.62*leak['deg_C']/(243.12+leak['deg_C'])))

In [ ]:
train2 = all_data[:len(train)]
test2 = all_data[len(train):]

train2 = pd.concat([train2, leak]).reset_index()
train2.drop(columns = 'index', inplace = True)

In [ ]:
def log_scaling(col):
  col = np.log1p(col)
  return col
  
cols = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']
for col in cols:
  train2[col] = log_scaling(train2[col])

In [ ]:
fig, ax = plt.subplots(len(cols), 2, figsize=(12,12))
n = 0
for i in cols:
  sns.histplot(train[i], ax=ax[n, 0]);
  sns.histplot(train2[i], ax = ax[n, 1]);
  n += 1

fig.tight_layout()
plt.show()

In [ ]:
train3 = train2.drop(columns = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])
test3 = test2.drop(columns = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])

In [ ]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import Ridge

cat = CatBoostRegressor(verbose = 200)
rf = RandomForestRegressor(n_jobs = -1, random_state = 42, n_estimators=1000)
lgbm = LGBMRegressor(n_jobs = -1, random_state = 42,
                     boosting_type = 'gbdt',
                    objective = 'regression',
                    metric = 'l2',
                    learning_rate = 0.1,
                    num_leaves = 63,
                    n_estimators = 1000,
                     max_depth = 8)
rid = Ridge(alpha = 100)

In [ ]:
def Cat_OOF(target, fold):

  print('----', target, '----')

  kf = KFold(n_splits=fold)
  result = []
  n = 1

  for train_index, test_index in kf.split(train3):
    print('Fold -', n)
    print("TRAIN:", train_index, "TEST:", test_index)

    X_train, X_test = train3.iloc[train_index], train3.iloc[test_index]
    y_train, y_test = train2[target][train_index], train2[target][test_index]
    cat.fit(X_train, y_train, verbose = False)

    result.append(np.exp(cat.predict(test3)) - 1)
    n += 1

  print('done!')

  return sum(result[x] for x in range(fold))/fold

In [ ]:
def LGBM_OOF(target, fold):

  print('----', target, '----')

  kf = KFold(n_splits=fold)
  result = []
  n = 1

  for train_index, test_index in kf.split(train3):
    print('Fold -', n)
    print("TRAIN:", train_index, "TEST:", test_index)

    X_train, X_test = train3.iloc[train_index], train3.iloc[test_index]
    y_train, y_test = train2[target][train_index], train2[target][test_index]
    lgbm.fit(X_train, y_train)

    result.append(np.exp(lgbm.predict(test3)) - 1)
    n += 1

  print('done!')
  
  return sum(result[x] for x in range(fold))/fold

In [ ]:
result_ca = Cat_OOF('target_carbon_monoxide', 5)
result_be = Cat_OOF('target_benzene', 5)
result_ni = Cat_OOF('target_nitrogen_oxides', 5)

In [ ]:
result_lgbm_ca = LGBM_OOF('target_carbon_monoxide', 5)
result_lgbm_be = LGBM_OOF('target_benzene', 5)
result_lgbm_ni = LGBM_OOF('target_nitrogen_oxides', 5)

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')
sub['target_benzene'] = result_be * 0.5 + result_lgbm_be * 0.5
sub['target_carbon_monoxide'] = result_ca * 0.5 + result_lgbm_ca * 0.5
sub['target_nitrogen_oxides'] = result_ni * 0.5 + result_lgbm_ni * 0.5
sub.to_csv('sub.csv', index = False)